In [2]:
import os
import glob

import pandas as pd
import numpy as np

### Gather all .h5 files

In [3]:
dirpath = '/raid/antoloui/Master-thesis/Data/Embeddings/test/'

# Get the names of all files in repo.
filepaths = glob.glob(dirpath + '*.h5')
for file in filepaths:
    df_file = pd.read_hdf(file)

['/raid/antoloui/Master-thesis/Data/Embeddings/test/dev01.h5',
 '/raid/antoloui/Master-thesis/Data/Embeddings/test/dev02.h5',
 '/raid/antoloui/Master-thesis/Data/Embeddings/test/dev00.h5']

In [ ]:
# Open dataframe.
df = 
df